In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
import pandas as pd
import os
import cv2

## Part 1: Data Preprocessing

### Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
# 設定資料路徑
# training data path
trcloudy='/content/drive/MyDrive/weather_image/train/cloudy/'
trrain='/content/drive/MyDrive/weather_image/train/rain/'
trshine='/content/drive/MyDrive/weather_image/train/shine/'
trsunrise='/content/drive/MyDrive/weather_image/train/sunrise/'
# testing data path
testpath='/content/drive/MyDrive/weather_image/test/'

In [ ]:
"""for i in range(len(os.listdir(trcloudy))):
  f=os.listdir(trcloudy)[i]
  img=cv2.imread(trcloudy+f)
  img=data_augmentation(img)
  img = np.float32(img)
  cv2.imwrite(trcloudy + str(i) + 'augmentation.jpg', img)
for i in range(len(os.listdir(trrain))):
  f=os.listdir(trrain)[i]
  img=cv2.imread(trrain+f)
  img=data_augmentation(img)
  img = np.float32(img)
  cv2.imwrite(trrain + str(i) + 'augmentation.jpg', img)
for i in range(len(os.listdir(trshine))):
  f=os.listdir(trshine)[i]
  img=cv2.imread(trshine+f)
  img=data_augmentation(img)
  img = np.float32(img)
  cv2.imwrite(trshine + str(i) + 'augmentation.jpg', img)
for i in range(len(os.listdir(trsunrise))):
  f=os.listdir(trsunrise)[i]
  img=cv2.imread(trsunrise+f)
  img=data_augmentation(img)
  img = np.float32(img)
  cv2.imwrite(trsunrise + str(i) + 'augmentation.jpg', img)"""

In [ ]:
# 顯示資料筆數
print('train cloudy length: ',len(os.listdir(trcloudy)))
print('train rain length: ',len(os.listdir(trrain)))
print('train shine length: ',len(os.listdir(trshine)))
print('train sunrise length: ',len(os.listdir(trsunrise)))
print('\n')
print('test data length: ',len(os.listdir(testpath)))

### convert image data to numpy

In [ ]:
# convert the train data to numpy
tr_label=list()
tr_data=np.empty(shape=(2096,100,100,3))
x=0
size = (100, 100)
for i in range(len(os.listdir(trcloudy))):
  f=os.listdir(trcloudy)[i]
  img=cv2.imread(trcloudy+f)
  img=cv2.resize(img,size)
  img=img[:,:,::-1]/255
  tr_data[x]=img
  tr_label.append(0)
  x+=1
for i in range(len(os.listdir(trrain))):
  f=os.listdir(trrain)[i]
  img=cv2.imread(trrain+f)
  img=cv2.resize(img,size)
  img=img[:,:,::-1]/255
  tr_data[x]=img
  tr_label.append(1)
  x+=1
for i in range(len(os.listdir(trshine))):
  f=os.listdir(trshine)[i]
  img=cv2.imread(trshine+f)
  img=cv2.resize(img,size)
  img=img[:,:,::-1]/255
  tr_data[x]=img
  tr_label.append(2)
  x+=1
for i in range(len(os.listdir(trsunrise))):
  f=os.listdir(trsunrise)[i]
  img=cv2.imread(trsunrise+f)
  img=cv2.resize(img,size)
  img=img[:,:,::-1]/255
  tr_data[x]=img
  tr_label.append(3)
  x+=1

tr_label=np.array(tr_label)

In [ ]:
print(tr_label.shape)

In [ ]:
# convert the test data to numpy
te_filename=list()
te_data=np.empty(shape=(75,100,100,3))
for i in range(len(os.listdir(testpath))):
  f=os.listdir(testpath)[i]
  te_filename.append(f)
  img=cv2.imread(testpath+f)
  img=cv2.resize(img,(100,100))
  img=img[:,:,::-1]/255
  te_data[i]=img

In [ ]:
# show data size
print('train data size:',tr_data.shape)
print('train label size:',tr_label.shape)
print('test data size:',te_data.shape)

### shuffle

In [ ]:
# on-hot label
from tensorflow.keras.utils import to_categorical
tr_label=to_categorical(tr_label)
# 打亂資料順序
from sklearn.utils import shuffle
tr_data,tr_label=shuffle(tr_data,tr_label,random_state=0)

### train_validation data split

In [ ]:
# train_validation data split
#val_data數量固定262個
train_data=tr_data[0:1836]
val_data=tr_data[1836:]
train_label=tr_label[0:1836]
val_label=tr_label[1836:]
print('train data size: ',train_data.shape)
print('validation data size: ',val_data.shape)
print('train label size: ',train_label.shape)
print('validation label size: ',val_label.shape)

## Part 2: Build a simple CNN

In [ ]:
# define model
cnn=models.Sequential() # name the network
# feature extraction
"""cnn.add(layers.Conv2D(filters=32,kernel_size=(3,3), input_shape=(300,300,3),activation='relu',padding='same'))
cnn.add(layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'))
cnn.add(layers.MaxPooling2D(pool_size=(2,2),padding='same'))
cnn.add(layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
cnn.add(layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))
cnn.add(layers.MaxPooling2D(pool_size=(2,2),padding='same'))
cnn.add(layers.Flatten())
# neron network
cnn.add(layers.Dense(units=64,activation='relu'))
cnn.add(layers.Dense(units=32,activation='relu'))
cnn.add(layers.Dense(units=4,activation='softmax'))"""

# feature extraction
cnn.add(layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(100,100,3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
cnn.add(layers.Dropout(0.25))  # Add dropout here

cnn.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
cnn.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
cnn.add(layers.Dropout(0.25))  # Add dropout here

cnn.add(layers.Flatten())

# neural network
cnn.add(layers.Dense(units=64, activation='relu'))
cnn.add(layers.Dropout(0.5))  # Add dropout here
cnn.add(layers.Dense(units=32, activation='relu'))
cnn.add(layers.Dropout(0.5))  # Add dropout here
cnn.add(layers.Dense(units=4, activation='softmax'))

# show the model structure
cnn.summary()

In [ ]:
# comiple model
cnn.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

class_weights = {0: 1.40703518,
                1: 1.,
                2: 1.16582915,
                3: 1.69346734}

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

In [ ]:
from sklearn.utils import class_weight
#history = cnn.fit(train_data, train_label,batch_size=10 ,epochs=100,validation_split=0.3)
history = cnn.fit_generator(datagen.flow(train_data, train_label, batch_size=32), validation_data=(val_data, val_label), epochs=100, class_weight=class_weights)

In [ ]:
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.title('loss curve')
plt.ylabel('loss')
plt.legend()
plt.show()
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='val_accuracy')
plt.title('accuracy curve')
plt.ylabel('accuracy')
plt.legend()
plt.show()

### Plot the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
pre=cnn.predict(val_data)
pre=np.argmax(pre,axis=1)

In [ ]:
import seaborn as sn
cm=confusion_matrix(pre,np.argmax(val_label,axis=1))
fit=plt.figure(figsize=(8,6))
plt.title('confusion matrix')
sn.heatmap(cm,annot=True,cmap='OrRd',fmt='g')
plt.xlabel('prediction')
plt.ylabel('true label')
plt.show()

# Output the csv file

In [ ]:
import pandas as pd

In [ ]:
prediction=cnn.predict(te_data)
prediction=np.argmax(prediction,axis=1)
prediction

In [ ]:
test_label=pd.DataFrame()
test_label['image_id']=te_filename
test_label['labels']=prediction
test_label=test_label.sort_values(by='image_id')
test_label.to_csv('/content/drive/MyDrive/weather_image/predict_label.csv',index=False) #結果轉csv檔

# Part3: Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.5),
])

In [ ]:
data=tr_data[1]
data=data_augmentation(data)
plt.imshow(data)